# Data engineering capstone - Eleflow

#### Instalando dependências

In [3]:
!pip install pandas requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/ae/e5/8c290f1dd50aae55d1ec20420a6df3c051d6f5ad78ee5b88b1a7ef26634b/charset_normalizer-3.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.0 MB/s eta 0:00:00
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/d2/b2/b157855192a68541a91ba7b2bbcb91f1b4faa51f8bae38d8005c034be524/urllib3-2.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━

#### imports necessários

In [46]:
import os
import json
import pandas as pd

## Carregar os dados de VRA
- Normalizar o cabeçalho para snake case
- Salvar estes dados

In [47]:
# Função para percorrer o diretório, pegar todos os arquivos e carregar as informações
def ler_jsons(diretorio):
    arquivos_json = [arquivo for arquivo in os.listdir(diretorio) if arquivo.endswith('.json')]

    for nome_arquivo in arquivos_json:
        caminho_arquivo = os.path.join(diretorio, nome_arquivo)
        with open(caminho_arquivo, 'r', encoding='utf-8-sig') as arquivo:
            yield json.load(arquivo)

In [48]:
# Nome das colunas em snake_case
nomes_colunas_vra = ['icao_empresa_aerea', 'numero_voo', 'codigo_autorizacao', 'codigo_tipo_linha',
                     'icao_aerodromo_origem', 'icao_aerodromo_destino', 'partida_prevista',
                     'partida_real', 'chegada_prevista', 'chegada_real', 'situacao_voo', 'codigo_justificativa']


In [49]:
# Diretório onde estão os arquivos JSON
diretorio_vra = './data_engineer_test/VRA'

In [50]:
# Criar um gerador
vra_generator = ler_jsons(diretorio_vra)

In [51]:
# Criação do dataframe com os dados de vra
vra_df = pd.DataFrame()

In [52]:
# Adicionando informações de cada arquivo JSON no dataframe
for lista_de_dicts in vra_generator:
    vra_df = vra_df._append(lista_de_dicts, ignore_index=True)

In [53]:
# Modificando nome das colunas do dataframe VRA
vra_df.columns = nomes_colunas_vra

In [54]:
print(vra_df.head(5))

  icao_empresa_aerea numero_voo codigo_autorizacao codigo_tipo_linha  \
0                AZU       4478                  0                 N   
1                AZU       4479                  0                 N   
2                AZU       4479                  0                 N   
3                AZU       4479                  0                 N   
4                AZU       4479                  0                 N   

  icao_aerodromo_origem icao_aerodromo_destino     partida_prevista  \
0                  SBRJ                   SBGO  2021-06-30 22:15:00   
1                  SBGO                   SBRJ  2021-06-07 06:20:00   
2                  SBGO                   SBRJ  2021-06-08 06:20:00   
3                  SBGO                   SBRJ  2021-06-09 06:20:00   
4                  SBGO                   SBRJ  2021-06-10 06:20:00   

          partida_real     chegada_prevista         chegada_real situacao_voo  \
0  2021-06-30 22:00:00  2021-06-30 23:55:00  2021-06-30 23:

## Carregar dos dados de AIR_CIA
- Normalizar o cabeçalho para snake case
- Separar a coluna 'ICAO IATA' em duas colunas, seu conteúdo está separado por espaço e pode não conter o código IATA, caso não contenha o código IATA, deixe o valor nulo.
- Salvar estes dados

In [63]:
# Diretório onde estão os arquivos CSV
diretorio_air_cia = './data_engineer_test/AIR_CIA/'

In [64]:
# Normalizando as colunas para snake_case
nomes_colunas_air_cia = ['razao_social', 'icao_iata', 'cnpj', 'atividades_aereas',
                         'endereco_sede', 'telefone', 'email', 'decisao_operacional', 
                         'data_decisao_operacional', 'validade_operacional']

In [65]:
# Inicializa um DataFrame vazio para armazenar os dados
df_air_cia = pd.DataFrame()

In [66]:
# Liste todos os arquivos CSV na pasta
for filename in os.listdir(diretorio_air_cia):
    if filename.endswith(".csv"):
        # Crie o caminho completo para o arquivo
        arquivo = os.path.join(diretorio_air_cia, filename)
        
        # Leia o arquivo CSV e adicione os dados ao DataFrame total
        df = pd.read_csv(arquivo, sep=';')
        df_air_cia = pd.concat([df_air_cia, df], ignore_index=True)

In [67]:
# Modificando nome das colunas do dataframe AIR_CIA
df_air_cia.columns = nomes_colunas_air_cia


In [68]:
# separando os valores pelo espaço em icao_iata e colocando nas novas colunas: icao e iata
df_air_cia[['icao', 'iata']] = df_air_cia['icao_iata'].str.split(n=1, expand=True)

In [69]:
# Removendo icao_iata e deixando as novas colunas na mesma posição que ela estava 
posicao_icao_iata = nomes_colunas_air_cia.index('icao_iata')
nomes_colunas_air_cia.pop(posicao_icao_iata)
nomes_colunas_air_cia.insert(posicao_icao_iata, 'icao')
nomes_colunas_air_cia.insert(posicao_icao_iata + 1, 'iata')
df_air_cia = df_air_cia[nomes_colunas_air_cia]

In [70]:
print(df_air_cia.head(5))

                                        razao_social icao  iata  \
0  AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...  ASO    2S   
1                        ATA - AEROTÁXI ABAETÉ LTDA.  ABJ  None   
2                               OMNI TÁXI AÉREO S.A.  OMI    O1   
3                                PRIME TÁXI AIR LTDA  NaN   NaN   
4                        SIDERAL LINHAS AÉREAS LTDA.  SID    0S   

                 cnpj                                  atividades_aereas  \
0  27.315.694/0001-02  SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...   
1  14.674.451/0001-19  TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...   
2  03.670.763/0001-38  AEROFOTOGRAFIA, AEROINSPEÇÃO, SERVIÇOS AÉREOS ...   
3  23.568.370/0001-25  TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, SERV...   
4  10.919.908/0001-57  TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...   

                                       endereco_sede  \
0  RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...   
1   PÇA. GAGO COUTINHO, S/N,

## Criar nova tabela aerodromos
- Através da API https://rapidapi.com/Active-api/api/airport-info/ trazer os aeródramos através do código ICAO presente nos dados de VRA.
- Salvar estes dados
